In [7]:
# test_identity_onnx.py
import cv2
from models.identity_verifier_onnx import IdentityVerifierONNX
from models.liveness_model import LivenessDetector
from utils.config import IDENTITY_MODEL, LIVENESS_MODEL

print("Loading models...")
verifier = IdentityVerifierONNX(IDENTITY_MODEL)
liveness = LivenessDetector(LIVENESS_MODEL)

print("\n✅ Webcam test: Take 2 photos of yourself")
print("Press 'C' to capture first photo")
print("Press 'C' again to capture second photo")
print("Press 'Q' to quit\n")

cap = cv2.VideoCapture(0)
photos = []

while len(photos) < 2:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Detect face
    live_result = liveness.predict(frame)
    
    # Draw box
    if live_result['face_detected']:
        bbox = live_result['bbox']
        cv2.rectangle(frame, tuple(bbox[0]), tuple(bbox[1]), (0, 255, 0), 2)
        cv2.putText(frame, f"Photo {len(photos) + 1}/2 - Press 'C'", 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    cv2.imshow("Identity Verification Test", frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('c') and live_result['face_detected']:
        # Capture face crop
        bbox = live_result['bbox']
        face_crop = frame[bbox[0][1]:bbox[1][1], bbox[0][0]:bbox[1][0]]
        photos.append(face_crop)
        print(f"✅ Captured photo {len(photos)}")
    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

if len(photos) == 2:
    print("\n🔍 Verifying identity...")
    result = verifier.verify_identity(photos[0], photos[1], threshold=0.70)
    
    print(f"\n{'='*50}")
    print(f"Match Score: {result['match_score']:.2%}")
    print(f"Same Person: {result['is_same_person']}")
    print(f"Threshold: {result['threshold_used']:.0%}")
    print(f"{'='*50}")
    
    if result['is_same_person']:
        print("✅ IDENTITY VERIFIED - Same person detected!")
    else:
        print("❌ IDENTITY MISMATCH - Different person or poor quality!")
else:
    print("⚠️  Test cancelled or insufficient photos captured")


Loading models...
✅ IdentityVerifierONNX loaded (InceptionResnetV1)
✅ LivenessDetector loaded (DeepPixBiS - Optimized)

✅ Webcam test: Take 2 photos of yourself
Press 'C' to capture first photo
Press 'C' again to capture second photo
Press 'Q' to quit

✅ Captured photo 1
✅ Captured photo 2

🔍 Verifying identity...

Match Score: 98.85%
Same Person: True
Threshold: 70%
✅ IDENTITY VERIFIED - Same person detected!


In [10]:
"""
Real-time eKYC demo with all components
Run: python realtime_demo.py
"""

import cv2
import time
import numpy as np
from models.deepfake_model import DeepfakeDetector
from models.liveness_model import LivenessDetector
from models.identity_verifier_onnx import IdentityVerifierONNX
from config import *

# Load models
print("Loading models...")
deepfake = DeepfakeDetector(DEEPFAKE_MODEL, device=DEVICE)
liveness = LivenessDetector(LIVENESS_MODEL)
identity = IdentityVerifierONNX(IDENTITY_MODEL, use_gpu=(DEVICE=='cuda'))

print("✅ All models loaded\n")
print("Controls:")
print("  'R' - Register reference face (for identity matching)")
print("  'Q' - Quit")
print("-" * 50)

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 30)

# FPS tracking
fps_history = []
reference_embedding = None

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    start_time = cv2.getTickCount()
    canvas = frame.copy()
    h, w = canvas.shape[:2]
    
    # Liveness detection
    live_result = liveness.predict(frame)
    
    if live_result['face_detected']:
        bbox = live_result['bbox']
        face_crop = frame[bbox[0][1]:bbox[1][1], bbox[0][0]:bbox[1][0]]
        
        # Deepfake detection
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        deep_result = deepfake.predict(frame_rgb)
        
        # Identity verification (if reference registered)
        identity_matched = False
        match_score = 0.0
        
        if reference_embedding is not None and face_crop.size > 0:
            current_embedding = identity.get_embedding(face_crop)
            if current_embedding is not None:
                match_score = identity.compute_similarity(reference_embedding, current_embedding)
                identity_matched = match_score >= 0.70
        
        # Overall verification
        verified = (
            deep_result['prediction'] == 'REAL' and
            live_result['is_live'] and
            (reference_embedding is None or identity_matched)
        )
        
        # Draw bounding box
        color = (0, 255, 0) if verified else (0, 0, 255)
        cv2.rectangle(canvas, tuple(bbox[0]), tuple(bbox[1]), color, 3)
        
        # Status overlay
        status_text = "✓ VERIFIED" if verified else "✗ REJECTED"
        cv2.putText(canvas, status_text, (10, 40),
                   cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 3)
        
        # Detailed info - ⚠️ FIX: Use 'real_probability' instead of 'confidence'
        y_offset = 80
        cv2.putText(canvas, f"Deepfake: {deep_result['prediction']} ({deep_result['real_probability']:.1%})",
                   (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        y_offset += 30
        
        cv2.putText(canvas, f"Liveness: {live_result['score']:.3f} {'✓' if live_result['is_live'] else '✗'}",
                   (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        y_offset += 30
        
        if reference_embedding is not None:
            cv2.putText(canvas, f"Identity: {match_score:.2%} {'✓' if identity_matched else '✗'}",
                       (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        else:
            cv2.putText(canvas, "Identity: No reference (Press 'R')",
                       (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 165, 0), 2)
    
    # FPS calculation
    elapsed = (cv2.getTickCount() - start_time) / cv2.getTickFrequency()
    fps = 1.0 / elapsed if elapsed > 0 else 0
    fps_history.append(fps)
    if len(fps_history) > 30:
        fps_history.pop(0)
    
    avg_fps = np.mean(fps_history)
    cv2.putText(canvas, f"FPS: {avg_fps:.1f}", 
               (w - 150, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    
    cv2.imshow("eKYC Real-time Demo", canvas)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('r') and live_result['face_detected']:
        # Register reference face
        bbox = live_result['bbox']
        face_crop = frame[bbox[0][1]:bbox[1][1], bbox[0][0]:bbox[1][0]]
        reference_embedding = identity.get_embedding(face_crop)
        if reference_embedding is not None:
            print("✅ Reference face registered!")
        else:
            print("❌ Failed to register face")

cap.release()
cv2.destroyAllWindows()
print("\n✅ Demo closed")


Loading models...
Loaded pretrained weights for efficientnet-b0
✅ DeepfakeDetector loaded on cpu with Grad-CAM explainability
✅ LivenessDetector loaded (DeepPixBiS - Optimized)
✅ IdentityVerifierONNX loaded (InceptionResnetV1)
✅ All models loaded

Controls:
  'R' - Register reference face (for identity matching)
  'Q' - Quit
--------------------------------------------------
✅ Reference face registered!
✅ Reference face registered!

✅ Demo closed


In [1]:
"""
Test enhanced explainability system
"""

from PIL import Image
from models.deepfake_model import DeepfakeDetector
from models.enhanced_explainability import EnhancedExplainability
from frequency_analyzer import FrequencyAnalyzer
from explainability_dashboard import save_dashboard_report
from config import DEEPFAKE_MODEL, DEVICE

print("="*60)
print("ENHANCED EXPLAINABILITY TEST")
print("="*60 + "\n")

# Load models
print("Loading models...")
base_detector = DeepfakeDetector(DEEPFAKE_MODEL, device=DEVICE)
enhanced_detector = EnhancedExplainability(base_detector)
frequency = FrequencyAnalyzer(device='cpu')
print()

# Load test image
test_img_path = input("Enter path to test image (or press Enter for webcam capture): ").strip()

if not test_img_path:
    # Capture from webcam
    import cv2
    print("\n📷 Opening webcam... Press 'C' to capture, 'Q' to quit")
    cap = cv2.VideoCapture(0)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        cv2.putText(frame, "Press 'C' to capture image", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.imshow("Capture Test Image", frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('c'):
            test_img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            print("✅ Image captured!")
            break
        elif key == ord('q'):
            print("❌ Cancelled")
            exit()
    
    cap.release()
    cv2.destroyAllWindows()
else:
    test_img = Image.open(test_img_path)

print("\n🔍 Running enhanced analysis...\n")

# Run predictions
pred = enhanced_detector.predict_with_enhanced_explainability(test_img)
freq = frequency.analyze_deepfake_frequency(test_img)

# Print results
print("="*60)
print(f"VERDICT: {pred['prediction']}")
print("="*60)
print(f"\nAuthenticity:")
print(f"  Real Probability: {pred['real_probability']:.1%}")
print(f"  Fake Probability: {pred['fake_probability']:.1%}")

if pred['prediction'] == 'FAKE':
    print(f"\nForgery Analysis:")
    print(f"  Type: {pred['forgery_type']}")
    print(f"  Description: {pred['forgery_description']}")
    print(f"  Confidence: {pred['forgery_confidence']:.1%}")

if pred['suspicious_regions']:
    print(f"\nSuspicious Regions Detected:")
    for region in pred['suspicious_regions']:
        print(f"  • {region['region']}: {region['confidence']:.1f}% activation")

if freq['is_suspicious']:
    print(f"\nFrequency Anomalies:")
    print(f"  Anomaly Score: {freq['anomaly_score']:.1%}")

print("\n" + "="*60)

# Generate and save dashboard
print("\n📊 Generating visual report...")
dashboard = save_dashboard_report(test_img, pred, "enhanced_report.png", freq)
dashboard.show()

print("\n✅ Test complete!")
print("Report saved to: enhanced_report.png")


ImportError: cannot import name 'DeepfakeDetector' from 'models.deepfake_model' (C:\Users\hp\Saved Games\ekyc_system\models\deepfake_model.py)

In [8]:
# test_liveness_fix.py
import cv2
from models.liveness_model import LivenessDetector
from config import LIVENESS_MODEL

print("Loading liveness detector...")
detector = LivenessDetector(LIVENESS_MODEL)

cap = cv2.VideoCapture(0)
print("✅ Press 'Q' to quit")
print("✅ Liveness threshold: >0.03 (GitHub standard)\n")

frame_count = 0
import time
start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    result = detector.predict(frame)
    frame_count += 1
    
    # Calculate FPS
    elapsed = time.time() - start_time
    fps = frame_count / elapsed if elapsed > 0 else 0
    
    print(f"Face: {result['face_detected']} | Live: {result['is_live']} | "
          f"Score: {result['score']:.4f} | FPS: {fps:.1f}")
    
    if result['face_detected']:
        bbox = result['bbox']
        color = (0, 255, 0) if result['is_live'] else (0, 0, 255)
        cv2.rectangle(frame, tuple(bbox[0]), tuple(bbox[1]), color, 2)
        
        text = f"{'LIVE' if result['is_live'] else 'SPOOF'} ({result['score']:.4f})"
        cv2.putText(frame, text, (bbox[0][0], bbox[0][1] - 10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        # Add FPS counter
        cv2.putText(frame, f"FPS: {fps:.1f}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    cv2.imshow("Liveness Test", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Loading liveness detector...
✅ LivenessDetector loaded
✅ Press 'Q' to quit
✅ Liveness threshold: >0.03 (GitHub standard)

Face: True | Live: False | Score: 0.2329 | FPS: 0.4
Face: True | Live: False | Score: 0.1205 | FPS: 0.7
Face: True | Live: True | Score: 0.5592 | FPS: 0.9
Face: True | Live: False | Score: 0.0138 | FPS: 1.1
Face: True | Live: False | Score: 0.0180 | FPS: 1.3
Face: False | Live: False | Score: 0.0000 | FPS: 1.6
Face: False | Live: False | Score: 0.0000 | FPS: 1.8
Face: False | Live: False | Score: 0.0000 | FPS: 2.1
Face: False | Live: False | Score: 0.0000 | FPS: 2.3
Face: False | Live: False | Score: 0.0000 | FPS: 2.5
Face: False | Live: False | Score: 0.0000 | FPS: 2.7
Face: False | Live: False | Score: 0.0000 | FPS: 2.9
Face: False | Live: False | Score: 0.0000 | FPS: 3.2
Face: True | Live: False | Score: 0.0127 | FPS: 3.2
Face: True | Live: False | Score: 0.1534 | FPS: 3.2
Face: True | Live: False | Score: 0.0978 | FPS: 3.2
Face: True | Live: False | Score: 0.087